---

## 1. Učitavanje potrebnih biblioteka i učitavanje podataka

Prvo ćemo učitati sve potrebne biblioteke za analizu podataka:

In [50]:
# Učitavanje potrebnih biblioteka
import pandas as pd         
import numpy as np           

# Prikaži verzije biblioteka
print("Verzije biblioteka:")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

# Učitavanje podataka iz CSV datoteke iz dataset mape
# read_csv je standardna Pandas funkcija za učitavanje podataka iz CSV formata
df_original = pd.read_csv('../dataset/student-por-G1-leq-10.csv')

# Kreiramo radnu kopiju s kojom ćemo raditi (backup originala u memoriji)
df = df_original.copy()

print("Datoteka uspješno učitana!")
print(f"\nDimenzije datoteke: {df.shape}")
print(f"  - Broj redaka (zapisa): {df.shape[0]}")
print(f"  - Broj stupaca (atributa): {df.shape[1]}")

Verzije biblioteka:
Pandas: 2.3.3
NumPy: 2.4.0
Datoteka uspješno učitana!

Dimenzije datoteke: (252, 33)
  - Broj redaka (zapisa): 252
  - Broj stupaca (atributa): 33


### Prikaz Osnovnih Informacija o DataFrameu

**Što prikazujemo:**
- Nazive svih stupaca (atributa)
- Tipove podataka u svakom stupcu
- Broj ne-nedostajućih vrijednosti
- Memorijsku potrošnju

In [51]:
# Detaljne informacije o DataFrameu
print("INFORMACIJE O DATAFREMU - PRIJE OBRADE:\n")
df.info()

INFORMACIJE O DATAFREMU - PRIJE OBRADE:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      252 non-null    object
 1   sex         252 non-null    object
 2   age         252 non-null    int64 
 3   address     252 non-null    object
 4   famsize     252 non-null    object
 5   Pstatus     252 non-null    object
 6   Medu        252 non-null    int64 
 7   Fedu        252 non-null    int64 
 8   Mjob        251 non-null    object
 9   Fjob        252 non-null    object
 10  reason      252 non-null    object
 11  guardian    252 non-null    object
 12  traveltime  252 non-null    int64 
 13  studytime   252 non-null    int64 
 14  failures    252 non-null    int64 
 15  schoolsup   252 non-null    object
 16  famsup      252 non-null    object
 17  paid        252 non-null    object
 18  activities  252 non-null    object
 19  nursery  

### Prikaz Prvih Redaka

Pogledajmo kako izgledaju prvi redci dataseta:

In [52]:
# Prikazi prvih 10 redaka
print("PRVIH 10 REDAKA DATASETA:\n")
print(df.head(10))
print("\n" + "="*60 + "\n")

PRVIH 10 REDAKA DATASETA:

  school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4   at_home   teacher   
1     GP   F   17       U     GT3       T     1     1   at_home     other   
2     GP   F   17       U     GT3       A     4     4     other   teacher   
3     GP   F   15       U     GT3       T     2     1  services     other   
4     GP   M   17       U     GT3       T     3     2  services  services   
5     GP   M   16       U     LE3       T     2     2     other     other   
6     GP   F   15       R     GT3       T     2     4  services    health   
7     GP   F   16       U     GT3       T     2     2  services  services   
8     GP   M   15       U     GT3       T     4     4    health  services   
9     GP   M   15       U     LE3       T     4     4   teacher     other   

   ... famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0  ...      4        3      4     1     1      3     

---

## 2. Otkrivanje Nedostajućih Vrijednosti

Sada ćemo analizirati gdje su nedostajuće vrijednosti u našem datasetu. Pandas koristi `NaN` (Not a Number) za označavanje nedostajućih vrijednosti.


### Prikaz redaka s nedostajućim vrijednostima

`.isnull().any(axis=1)` provjerava je li bilo koja vrijednost u redu NaN (`any(axis=1)` znači "provjeravaj redove" - axis=1), što nam daje sve redake koji imaju barem jednu nedostajuću vrijednost.

In [55]:
# Pronađi sve redove koji imaju BAR JEDNU nedostajuću vrijednost
# .any(axis=1) provjerava po redcima (axis=1) - vraća True ako red ima bar jednu NaN vrijednost
rows_with_missing = df[df.isnull().any(axis=1)]

print(f"REDCI S NEDOSTAJUĆIM VRIJEDNOSTIMA ({len(rows_with_missing)} redaka):\n")
print(rows_with_missing)

REDCI S NEDOSTAJUĆIM VRIJEDNOSTIMA (1 redaka):

   school sex  age address famsize Pstatus  Medu  Fedu Mjob    Fjob  ...  \
13     GP   M   15       U     LE3       A     4     2  NaN  health  ...   

   famrel freetime  goout  Dalc  Walc health absences  G1 G2 G3  
13      5        5      5     3     4      5        4  10  9  9  

[1 rows x 33 columns]


---

## 3. Rekodiranje Nedostajućih Vrijednosti

Sada ćemo **zamijeniti sve nedostajuće vrijednosti (NaN)** sa stringom **"missing"**.

### Zašto rekodirati nedostajuće vrijednosti?

1. **Jasnoća** - lakše je razumjeti podatke kada je eksplicitno navedeno što je "nedostaje"
2. **Analiza** - "missing" je stringovna vrijednost koja se može analizirati kao kategorija
3. **Usporedba** - lakše se može pregledati koji stupac ima više nedostajućih vrijednosti
4. **Čuvanje podataka** - čuva se informacija da je vrijednost nedostajala, za razliku od brisanja redaka

### Metoda: fillna() - Zamjena nedostajućih vrijednosti

`.fillna('missing')` zamjenjuje sve NaN vrijednosti navedenom vrijednosti (u ovom slučaju stringom "missing"), što omogućava da nedostajuće vrijednosti postanu vidljive kao eksplicitne kategorije umjesto numeričkog NaN.

In [56]:
# Zamjena svih NaN vrijednosti sa stringom "missing"
# fillna() metoda zamjenjuje sve NaN vrijednosti navedenom vrijednosti

print("REKODIRANJE NEDOSTAJUĆIH VRIJEDNOSTI:\n")
print("Korištenje .fillna('missing')...\n")

df_cleaned = df.fillna('missing')

print("Rekodiranje uspješno. Sve NaN vrijednosti zamijenjene sa 'missing'.")
print("\n" + "="*60 + "\n")

REKODIRANJE NEDOSTAJUĆIH VRIJEDNOSTI:

Korištenje .fillna('missing')...

Rekodiranje uspješno. Sve NaN vrijednosti zamijenjene sa 'missing'.




### Provjera rezultata rekodiranja

Pogledajmo redake s nedostajućim vrijednostima nakon rekodiranja:

In [57]:
# Prikaži redake koji su prije imali nedostajuće vrijednosti
print("REDCI NAKON REKODIRANJA (koji su prije imali nedostajuće vrijednosti):\n")
print(rows_with_missing.index.tolist())
print("\nPrikaz tih redaka iz očišćenog dataseta:\n")
print(df_cleaned.loc[rows_with_missing.index])
print("\n" + "="*60 + "\n")

REDCI NAKON REKODIRANJA (koji su prije imali nedostajuće vrijednosti):

[13]

Prikaz tih redaka iz očišćenog dataseta:

   school sex  age address famsize Pstatus  Medu  Fedu     Mjob    Fjob  ...  \
13     GP   M   15       U     LE3       A     4     2  missing  health  ...   

   famrel freetime  goout  Dalc  Walc health absences  G1 G2 G3  
13      5        5      5     3     4      5        4  10  9  9  

[1 rows x 33 columns]




### Provjera da više NEMA NaN vrijednosti

In [58]:
# Provjera da više nema NaN vrijednosti
total_missing_after = df_cleaned.isnull().sum().sum()

print("VERIFIKACIJA REKODIRANJA:\n")
print(f"Nedostajućih vrijednosti u očišćenom datasetu: {total_missing_after}")
print("\nProvjeravamo nedostajuće vrijednosti po stupcima:\n")
print(df_cleaned.isnull().sum())
print("\n" + "="*60 + "\n")

VERIFIKACIJA REKODIRANJA:

Nedostajućih vrijednosti u očišćenom datasetu: 0

Provjeravamo nedostajuće vrijednosti po stupcima:

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64


